In [1]:
import os

In [2]:
%pwd

'c:\\Users\\hp\\OneDrive\\Bureau\\Forecast Pipeline\\Demand-Forecasting-Pipeline\\Teraflow-Forecast-Pipeline\\steps'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\hp\\OneDrive\\Bureau\\Forecast Pipeline\\Demand-Forecasting-Pipeline\\Teraflow-Forecast-Pipeline'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelTrainerConfig_XGBoost:
    root_dir: Path
    model_dir_name: str
    X_train_data_path : Path
    y_train_data_path : Path
    X_test_data_path : Path
    y_test_data_path : Path
    model_name : str
    x_scaler_name : str
    y_scaler_name : str
    runtime_name : str
    n_estimators : int
    early_stopping_rounds : int

In [6]:
from src.mlProject.constants import *
from src.mlProject.utils.common import read_yaml, create_directories,save_json

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        
        create_directories([self.config.artifacts_root])
    
    def get_model_trainer_config_XGBoost(self) -> ModelTrainerConfig_XGBoost:
        config = self.config.model_trainer_xgboost
        params = self.params.XGBoost
        
        create_directories([config.root_dir])
        create_directories([os.path.join(config.root_dir,config.model_dir_name)])
        model_trainer_config = ModelTrainerConfig_XGBoost(
            root_dir = config.root_dir,
            model_dir_name = config.model_dir_name,
            X_train_data_path=config.X_train_data_path,
            y_train_data_path=config.y_train_data_path,
            X_test_data_path= config.X_test_data_path,
            y_test_data_path= config.y_test_data_path,
            model_name = config.model_name,
            x_scaler_name = config.x_scaler_name,
            y_scaler_name = config.y_scaler_name,
            n_estimators = params.n_estimators,
            early_stopping_rounds = params.early_stopping_rounds,
            runtime_name = config.runtime_name
        )
        return model_trainer_config

In [8]:

import pandas as pd
import os
from src.mlProject.utils import logger
from xgboost import XGBRegressor
import joblib
import numpy as np
from sklearn.preprocessing import StandardScaler
import time
from sklearn.model_selection import train_test_split


In [9]:
class ModelTrainer:
    def __init__(self,config:ModelTrainerConfig_XGBoost):
        self.config = config
        
    def load_scale_data(self):
        X_train =np.array(pd.read_csv(self.config.X_train_data_path))
        y_train =np.array(pd.read_csv(self.config.y_train_data_path))
        X_test = np.array(pd.read_csv(self.config.X_test_data_path))
        y_test = np.array(pd.read_csv(self.config.y_test_data_path))
        
        std_x = StandardScaler()
        X_train_scaled = std_x.fit_transform(X_train)
        X_test_scaled = std_x.transform(X_test)

        std_y = StandardScaler()
        y_train_scaled = std_y.fit_transform(y_train)
        y_test_scaled = std_y.transform(y_test)
        
        np.savetxt(os.path.join(os.path.join(self.config.root_dir,self.config.model_dir_name),"X_test_scaled.csv"), X_test_scaled, delimiter=',')
        np.savetxt(os.path.join(os.path.join(self.config.root_dir,self.config.model_dir_name),"y_test_scaled.csv"), y_test_scaled, delimiter=',')
        
        joblib.dump(std_x,os.path.join(os.path.join(self.config.root_dir,self.config.model_dir_name),self.config.x_scaler_name))
        joblib.dump(std_y,os.path.join(os.path.join(self.config.root_dir,self.config.model_dir_name),self.config.y_scaler_name))

        
        return X_train_scaled,y_train_scaled,
        
    def train(self,X_train_scaled,y_train_scaled): 
        
        X_train, X_val, y_train, y_val = train_test_split(X_train_scaled, y_train_scaled, test_size=0.2, random_state=42)
        xgb = XGBRegressor(n_estimators=self.config.n_estimators,early_stopping_rounds=self.config.early_stopping_rounds,eval_metric="rmse")
        
        start_xgboost = time.time()
        xgb.fit(X_train, y_train,eval_set=[(X_train, y_train), (X_val, y_val)],verbose=True)        
        end_xgboost = time.time()
        
        training_time = end_xgboost - start_xgboost
        training_time = {"training time" : training_time}
        save_json(path=Path(os.path.join(os.path.join(self.config.root_dir,self.config.model_dir_name),self.config.runtime_name)),data=training_time)        
        joblib.dump(xgb,os.path.join(os.path.join(self.config.root_dir,self.config.model_dir_name),self.config.model_name))

In [10]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config_XGBoost()
    model_trainer_config = ModelTrainer(config=model_trainer_config)
    X_train_scaled,y_train_scaled = model_trainer_config.load_scale_data()
    model_trainer_config.train(X_train_scaled,y_train_scaled)
except Exception as e:
    raise e

[2024-04-18 10:55:59,639: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-04-18 10:55:59,652: INFO: common: yaml file: params.yaml loaded successfully]
[2024-04-18 10:55:59,659: INFO: common: created directory at: artifacts]
[2024-04-18 10:55:59,662: INFO: common: created directory at: artifacts/model_trainer]
[2024-04-18 10:55:59,666: INFO: common: created directory at: artifacts/model_trainer\XGBoost]
[0]	validation_0-rmse:0.75127	validation_1-rmse:0.58487
[1]	validation_0-rmse:0.54224	validation_1-rmse:0.43271
[2]	validation_0-rmse:0.39400	validation_1-rmse:0.32517
[3]	validation_0-rmse:0.28829	validation_1-rmse:0.24926
[4]	validation_0-rmse:0.21251	validation_1-rmse:0.19580
[5]	validation_0-rmse:0.15769	validation_1-rmse:0.15911
[6]	validation_0-rmse:0.11773	validation_1-rmse:0.13468
[7]	validation_0-rmse:0.08848	validation_1-rmse:0.11908
[8]	validation_0-rmse:0.06702	validation_1-rmse:0.10948
[9]	validation_0-rmse:0.05123	validation_1-rmse:0.10358
[2024-04-1